# Deep Image Stack Viewer

<div style="border: 2px solid #ccc; border-radius: 8px; padding: 5px; display: inline-block;">
    <img src="assets/20240725_deep_image_stack_annotation.png" alt="Header Image" style="display: block; max-width: 100%;">
</div>


---

In [ ]:
from IPython.display import HTML
HTML("""
<div style="display: flex; justify-content: center; padding: 10px;">
    <iframe width="560" height="315" src="https://www.youtube.com/embed/Qa-wrIdMYH0?si=KDzApOEt2e4ROu-l" title="YouTube video player" frameborder="0" allow="accelerometer; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>
</div>
""")


## Prerequisites

| What? | Why? |
| --- | --- |
| [Index: Intro, Workflows, Extensions](./index.ipynb) | For context and workflow selection/feature guidance |
| [Deep Image Stack Workflow](./workflow_deep-image-stack.ipynb) | Recommended workflow |

## Overview

This workflow demonstrates how to work with deep image stacks, particularly those from microscopy data. We'll explore how to load, process, and visualize multi-dimensional image data while maintaining performance and interactivity. The workflow includes spatial annotation capabilities and dynamic plotting of timeseries based on annotated regions.

### Key Software

Alongside the core [HoloViz](https://github.com/holoviz) and [Bokeh](https://holoviz.org/) tools, we make extensive use of several key open source libraries:

- **[HoloNote](https://github.com/holoviz/holonote):** A newer HoloViz package that provides annotation capabilities
- **[Xarray](https://docs.xarray.dev/):** Handles multi-dimensional arrays with labeled coordinates
- **[NumPy](https://numpy.org/):** Provides the foundation for numerical computing in Python

---

## Imports and Configuration

In [ ]:
from pathlib import Path
import numpy as np
import xarray as xr
import holoviews as hv
from holoviews.operation.datashader import rasterize
import panel as pn
import fsspec

pn.extension('tabulator')
hv.extension('bokeh')

## Downloading the Data

Since this dataset isn't so big (~300 MB), let's download it so we can have a local copy and avoid any network delays.

<div class="admonition alert alert-warning">
    <p class="admonition-title" style="font-weight:bold">Warning</p>
    The following cell will download ~300 MB the first time it is run.
</div>

In [ ]:
DATA_URL = 'https://datasets.holoviz.org/miniscope/v1/real_miniscope_uint8.zarr/'
DATA_DIR = Path('./data')
DATA_FILENAME = Path(DATA_URL).name
DATA_PATH = DATA_DIR / DATA_FILENAME
print(f'Local Original Data Path: {DATA_PATH}')

DATA_DIR.mkdir(parents=True, exist_ok=True)

# Check if local directory exists
if not DATA_PATH.exists():
    print(f'Downloading data to: {DATA_PATH}')
    # Open the remote dataset and save it locally
    ds_remote = xr.open_dataset(
        fsspec.get_mapper(DATA_URL), engine='zarr', chunks={}
    )
    ds_remote.to_zarr(str(DATA_PATH))  # Save locally
    print(f'Dataset downloaded to: {DATA_PATH}')
else:
    print(f'Data already exists locally at: {DATA_PATH}')

## Loading and Inspecting the Data

Let's read the data in chunks, emulating a situation where a dataset is too large to fit into memory. Utilizing the `chunks` parameter in `xr.open_dataset` is crucial for efficient data handling with large datasets, as it enables Dask to process the data in manageable portions.

In [ ]:
# Open the dataset from the local copy
ds = xr.open_dataset(
    DATA_PATH.as_posix(),
    engine='zarr',
    chunks={'frame': 400, 'height': -1, 'width': -1}
)

# Access the variable 'varr_ref'
da = ds['varr_ref']
da

## Loading and Inspecting the Data

Let's start by loading our microscopy data. We'll use a sample dataset stored in the Zarr format, which is particularly well-suited for large, multi-dimensional arrays.

In [ ]:
# DATA_PATH = "data/real_miniscope_uint8.zarr"

# ds = xr.open_dataset(
#     DATA_PATH,
#     engine = 'zarr',
#     chunks = {'frame': 400, 'height':-1, 'width':-1},  # chunk by sets of complete frames
# )
# da = ds['varr_ref']
# da

## Data Visualization

We'll create a HoloViews-based application that allows for interactive exploration of the image stack. The app includes a player widget to control frame playback and a dynamic image display.


### Basic Image Stack Viewer


In [ ]:
# -----------------------------------------------------------------------------
# Image Visualization Functions
# -----------------------------------------------------------------------------


def plot_image(value):
    """
    Creates a HoloViews Image object for a single frame with custom styling and interactions

    Args:
        value: Frame number to display
    Returns:
        HoloViews Image with configured options for visualization
    """
    return hv.Image(
        da.sel(frame=value),  # Select the requested frame from our data
        kdims=["width", "height"],  # Specify dimensions for the image
    ).opts(
        # Basic display options
        title=f"frame = {value}",
        frame_height=da.sizes["height"],
        frame_width=da.sizes["width"],
        # Colormap settings
        cmap="Viridis",  # Use Viridis colormap for good perceptual properties
        clim=(0, 20),  # Set color scale limits
        colorbar=True,  # Show colorbar for intensity reference
        # Interactive features
        tools=["hover", "crosshair"],  # Enable hover tooltips and crosshair cursor
        toolbar="right",  # Position the toolbar
        apply_hard_bounds=True,  # Prevent panning outside data bounds
        # Scale bar for physical measurements
        scalebar=True,
        scalebar_unit=("µm", "m"),  # Set microscopy-appropriate units
        # Scale bar appearance
        scalebar_opts={
            "background_fill_alpha": 0.5,  # Semi-transparent background
            "border_line_color": None,  # No border
            "bar_length": 0.10,  # Length relative to plot
        },
    )


# -----------------------------------------------------------------------------
# Maximum Projection Overlay
# -----------------------------------------------------------------------------

# Create maximum intensity projection
max_proj_time = da.max("frame").compute().astype(np.float32)
img_max_proj_time = hv.Image(
    max_proj_time,  # Use max value across time dimension
    ["width", "height"],  # Spatial dimensions
    label="Max Over Time",  # Label for the overlay
).opts(
    cmap="magma",  # Different colormap to distinguish from main view
)

# -----------------------------------------------------------------------------
# Interactive Controls
# -----------------------------------------------------------------------------

# Video player for frame navigation
video_player = pn.widgets.Player(
    length=len(da.coords["frame"]),  # Total number of frames
    interval=100,  # 100ms between frames during playback
    value=250,  # Start at frame 250
    show_loop_controls=False,  # Hide loop controls for cleaner interface
    align="center",  # Center the player controls
    scale_buttons=0.9,  # Slightly smaller buttons
    sizing_mode="stretch_width",  # Fill available width
    show_value=True,  # Display current frame number
    value_align="center",  # Center the frame number
    visible_buttons=[  # Customize available controls
        "slower",
        "previous",
        "pause",
        "play",
        "next",
        "faster",
    ],
)

# Transparency slider for max projection overlay
alpha_slider = pn.widgets.FloatSlider(
    start=0,
    end=1,
    step=0.1,
    value=0.3,
    name="Opacity",
    align="center",
    sizing_mode="stretch_width",
)


# Create dynamic map that updates image based on player value
frame_dmap = hv.DynamicMap(pn.bind(plot_image, video_player.param.value_throttled))

# Link slider to overlay opacity
alpha_slider.jslink(img_max_proj_time, value="glyph.global_alpha")

# -----------------------------------------------------------------------------
# Layout Components
# -----------------------------------------------------------------------------

# Wrap controls in Card widgets for better organization
player_layout = pn.Card(
    video_player,
    title="Playback",
    sizing_mode="stretch_width",
    margin=(0, 0, 20, 0),
)

alpha_slider_layout = pn.Card(
    alpha_slider,
    title="Max Projection Overlay",
    sizing_mode="stretch_width",
    margin=(0, 0, 20, 0),
)

main_view = frame_dmap * img_max_proj_time
widgets = pn.Column(
    player_layout,
    alpha_slider_layout,
    align="center",
    width=350,
)

In [ ]:
# pn.Row(widgets, main_view) # uncomment to display the basic viewer

Here's a static snapshot of what the previous cell produces:

<div style="position: relative; display: inline-block;">
    <img src="./assets/deep_image_stack_anno_step1.png" alt="Static Preview Image Stack Player" style="display: block;">
    <div style="
        position: absolute;
        top: 0;
        left: 0;
        width: 100%;
        height: 100%;
        display: flex;
        align-items: center;
        justify-content: center;
        font-size: 40px;
        color: rgba(255, 255, 255, 0.5);
        background-color: rgba(0, 0, 0, 0.25);
        text-align: center;
        border-radius: 5px;
    ">
        Static Preview Image Stack Player
    </div>
</div>

## Adding side Views

In [ ]:
side_view_opts = dict(
    cmap = "greys_r",
    tools=['crosshair', 'hover'],
    axiswise=True,
    apply_hard_bounds=True,
    colorbar=False,
    toolbar=None,
)

top_data = da.mean(["height"]).persist()
top_view = rasterize(
    hv.Image(top_data, kdims=["width", "frame"]).opts(
        frame_height=175,
        frame_width=da.sizes['width'],
        title= "Top Side View",
        xaxis='top',
        **side_view_opts
    )
)

right_data = da.mean(["width"]).persist()
right_view = rasterize(
    hv.Image(right_data, kdims=["frame", "height"]).opts(
        frame_height=da.sizes['height'],
        title="Right Side View",
        yaxis='right',
        **side_view_opts
    )
)

In [ ]:
def plot_hline(value, x_range, y_range):
    if x_range == None:
        x_range = [int(da.width[0].values), int(da.width[-1].values)]
    return hv.Segments((x_range[0], value, x_range[1], value)).opts(axiswise=True)

def plot_vline(value, x_range, y_range):
    if y_range == None:
        y_range = [int(da.height[0].values), int(da.height[-1].values)]
    return hv.Segments((value, y_range[0], value, y_range[1])).opts(axiswise=True)

line_opts = dict(color="red", line_width=3, line_alpha=.4, line_dash='dashed')
xyrange_stream = hv.streams.RangeXY(source=main_view)
dmap_hline = hv.DynamicMap(pn.bind(plot_hline, video_player), streams=[xyrange_stream]).opts(**line_opts, **side_view_opts)
dmap_vline = hv.DynamicMap(pn.bind(plot_vline, video_player), streams=[xyrange_stream]).opts(**line_opts, **side_view_opts)

In [ ]:
top_view_overlay = (top_view * dmap_hline).opts(axiswise=True)
right_view_overlay = (right_view * dmap_vline).opts(axiswise=True)

## Timeseries of Spatial Annotation

To enable region-of-interest analysis, we'll first add spatial annotation capabilities using HoloNote. This allows users to mark and analyze specific regions within the image stack.


In [ ]:
from holonote.annotate import Annotator
from holonote.app import PanelWidgets
from holonote.app.tabulator import AnnotatorTable

In [ ]:
# -----------------------------------------------------------------------------
# Annotation Configuration
# -----------------------------------------------------------------------------

# Initialize annotator for marking regions of interest
annotator = Annotator(
    {"height": float, "width": float},  # Dimensions we can annotate
    fields=["type"],  # Additional fields to track
    groupby="type",  # Group annotations by type field
)

# Define color scheme for different annotation types
color_dim = hv.dim("type").categorize(
    categories={
        "A": "red",  # Type A annotations in red
        "B": "orange",  # Type B annotations in orange
        "C": "cyan",  # Type C annotations in cyan
    },
    default="grey",  # Use grey for any undefined types
)

# Apply styling to annotations
annotator.style.color = color_dim  # Color annotations by type
annotator.style.alpha = 0.3  # Make annotations semi-transparent

# -----------------------------------------------------------------------------
# Annotation Interface Components
# -----------------------------------------------------------------------------

# Create widget interface for annotation controls
panel_widgets = PanelWidgets(annotator)

# Create table for viewing and editing annotations
table_widget = AnnotatorTable(
    annotator,
    tabulator_kwargs={
        "sizing_mode": "stretch_width",  # Fill available width
        "theme": "midnight",  # Dark theme for contrast
        "layout": "fit_columns",  # Automatically size columns
        "sortable": False,  # Disable sorting
        "stylesheets": [":host .tabulator {font-size: 9px;}"],  # Compact font size
    },
)

# -----------------------------------------------------------------------------
# Layout
# -----------------------------------------------------------------------------

# Combine annotation controls and table into a single card
annotator_widgets = pn.Card(
    pn.Column(
        panel_widgets,  # Annotation control widgets at top
        table_widget,  # Table view below
    ),
    title="Annotator",
    sizing_mode="stretch_width",
    margin=(0, 0, 20, 0),  # Add bottom margin
    collapsed=False,
)


Now, we'll create a visualization that shows the mean timeseries for each annotated region, updating automatically as regions are added or modified.


In [ ]:
# -----------------------------------------------------------------------------
# Timeseries Visualization Options
# -----------------------------------------------------------------------------

# Default options for all timeseries curves
curve_opts = dict(
    responsive=True,  # Adjust to window size
    min_height=300,  # Minimum plot height
    max_height=600,  # Maximum plot height
    show_legend=False,  # Hide legend since we use colors
    xlabel="frame",  # X-axis label
    tools=["hover"],  # Enable hover tooltips
    line_alpha=0.5,  # Semi-transparent lines
    framewise=True,  # Reset ranges when data changes
    axiswise=True,  # Independent axes for subcoordinates
)

# Styling for the current frame indicator line
vline_opts = dict(color="grey", line_width=4, alpha=0.5)

# -----------------------------------------------------------------------------
# Dynamic Plotting Functions
# -----------------------------------------------------------------------------


def plot_ts(event):
    """
    Creates timeseries plots for each annotated region, showing mean intensity over time
    """
    curves = {}
    df = annotator.df

    # Process each annotated region
    for idx, row in df.iterrows():
        # Get region boundaries
        h1, h2, w1, w2 = row[
            ["start[height]", "end[height]", "start[width]", "end[width]"]
        ]

        # Calculate mean intensity for region
        da_sel = da.sel(height=slice(h1, h2), width=slice(w1, w2))
        mean_ts = da_sel.mean(["height", "width"])

        # Create identifiers for the curve
        group = f"G_{row['type']}"
        label = f"L_{idx[:6]}"

        # Create and style the curve
        curve = hv.Curve(mean_ts, group=group, label=label)
        curve = curve.opts(
            subcoordinate_y=True,  # Give each curve its own y-scale
            color=panel_widgets.colormap[row["type"]],  # Color by type
            **curve_opts,
        )
        curves[(group, label)] = curve

    # Update plot with new curves
    time_series.object = (vline * hv.Overlay(curves, kdims=["curve"])).opts(
        hv.opts.Curve(xlim=(frames[0], frames[-1])),
    )


def plot_frame_indicator_line(value):
    """Creates vertical line indicating current frame position"""
    if value:
        return hv.VSpans((value, value)).opts(
            axiswise=True, framewise=True, **vline_opts
        )


# -----------------------------------------------------------------------------
# Initialize Plot Components
# -----------------------------------------------------------------------------

# Get frame range for x-axis limits
frames = da.coords["frame"].values

# Create vertical line indicator that follows video player
vline = hv.DynamicMap(pn.bind(plot_frame_indicator_line, video_player)).opts(
    hv.opts.VLine(**vline_opts)
)

# Initialize empty timeseries plot
time_series = pn.pane.HoloViews(
    vline
    * hv.Curve([]).opts(
        xlim=(frames[0], frames[-1]),
        title="Create an annotation in the image",
        **curve_opts,
    )
)

# Connect annotation events to plotting function
annotator.on_event(plot_ts)

## Layout

In [ ]:
# -----------------------------------------------------------------------------
# Layout Assembly
# -----------------------------------------------------------------------------

# Combine image view components
main_view_overlay = main_view * annotator

# Add annotator UI to the widgets layout
widgets.append(annotator_widgets)

# Create main content layout
main_layout = pn.Column(
    top_view_overlay * annotator,
    pn.Row(main_view_overlay, right_view_overlay * annotator),
    time_series,  # Timeseries below
)

# Assemble final application layout
img_stack_app_annot = pn.Row(
    widgets,      # Controls on left  
    main_layout,  # Main content on right
    align='start',
)

# Display the application in this notebook
img_stack_app_annot


## Standalone App Extension

For deployment as a standalone web application, we can wrap our visualization in a Panel template:


In [ ]:
# -----------------------------------------------------------------------------
# Standalone Application Configuration
# -----------------------------------------------------------------------------

# Create a deployable web application using Panel's FastListTemplate
standalone_app = pn.template.FastListTemplate(
    # Application title shown in header
    title="Deep Image Stack App with Timeseries of Spatial Annotation",
    # Sidebar configuration
    sidebar=[widgets],  # Place all control widgets in sidebar
    sidebar_width=350,  # Set fixed width for sidebar
    # Main content area
    main=[main_layout],  # Place image view and timeseries in main area
    main_layout=None,  # Remove the background component to avoid color conflicts
    # Other styling
    theme="dark",
    accent="#30023f",
).servable()  # Make the app servable


To serve the standalone app, use `panel serve <path-to-this-file> --show` in your terminal while in the same conda environment.

<div class="admonition alert alert-warning">
    <p class="admonition-title" style="font-weight:bold">Warning</p>
    Clear notebook output and restart the kernel before serving the standalone application to avoid conflicts between notebook and served versions.
</div>

## What Next?
- Return to the [Index](./index.ipynb) page to explore other workflows and alternatives

## Related Resources

| What? | Why? |
| --- | --- |
| [Xarray Documentation](https://docs.xarray.dev/) | Learn about working with labeled multi-dimensional arrays |
| [HoloViz Documentation](https://holoviz.org/) | Comprehensive guide to the HoloViz ecosystem |